<a href="https://colab.research.google.com/github/Pugianf/Big_Data_and_Public_Sector_III-Economic_Development/blob/main/Trablaho_Final_Big_Data_e_Setor_P%C3%BAblico_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questão 1 - Biblioteca ipeadatapy e atualização statsmodel

In [ ]:
# instalação ipeadatapy e atualização statsmodels

from IPython.display import clear_output

!pip uninstall statsmodels -y
!pip install statsmodels
!pip install ipeadatapy
!pip install pybacen
!pip install wbgapi
!pip install geopandas
!pip install mapclassify
!pip install geobr

clear_output() 

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

clear_output()

In [ ]:
# importação de bibliotecas necessárias

# manipulação de dados

import pandas as pd
import numpy as np

# estatística e modelos

from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms

# gráficos

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick 
import matplotlib as mpl
import seaborn as sns

# mapas

import geopandas as gpd
import mapclassify
import geobr

# estilo dos gráficos

from matplotlib import style
style.use("ggplot")

# coleta de dados
import ipeadatapy
from pybacen.bacen import time_series 
import wbgapi as wb

# web scraping
import requests
from bs4 import BeautifulSoup

## Questão 2 - Montando DataFrame com ipeadatapy

Professor, favor notar que está ocorrendo falha nas bibliotecas 'ipeadatapy' e 'dadosbr', de onde se supunha buscar os dados. Dado que a falha ocorre em diferentes bibliotecas que tentam buscar os dados no IPEAData, creio que seja alguma modificação recente, ao menos momentânea, na API do site. Dessa forma, busquei os dados em csv (o qual encaminho) para poder utilizá-los no excercício. Os desafios para utilização dos dados foram apenas diversos dos pretendidos para a atividade.

In [ ]:
# verificando a base de dados

ipeadatapy.describe('PAN12_IPCAG12')

,Índice de Preços ao Consumidor Ampliado (IPCA)
Name,Índice de Preços ao Consumidor Ampliado (IPCA)
Code,PAN12_IPCAG12
Big Theme,Macroeconômico
Theme,Sinopse macroeconômica
Source,Instituto Brasileiro de Geografia e Estatístic...
Source acronym,IBGE/SNIPC
Comment,Índice Nacional de Preços ao Consumidor Amplo ...
Last update,2022-01-12T08:30:01.28-03:00
Frequency,Mensal
Measure,(% a.a.)


In [ ]:
# coletando taxa de inflação

dfIPCA = ipeadatapy.timeseries('PAN12_IPCAG12') 

dfIPCA

# conforme visto, tem sido apresentada inconsistência ao se solicitar a base de dados

In [ ]:
# tentando através de outro pacote

!pip install seriesbr

from seriesbr import ipea

In [ ]:
ipea.get_series("PAN12_IPCAG12")

In [ ]:
# utilizando csv baixado através da página http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=38391

dfIPCA = pd.read_csv("/content/drive/MyDrive/IDP/Dataset/PAN12_IPCAG12.CSV", sep=';', encoding='latin1')

In [ ]:
dfIPCA

,Data,Índice de Preços ao Consumidor Ampliado (IPCA)
0,1980/01,115.70
1,1980/02,71.88
2,1980/03,102.10
3,1980/04,85.55
4,1980/05,94.55
...,...,...
499,2021/08,10.95
500,2021/09,14.84
501,2021/10,16.07
502,2021/11,12.02


In [ ]:
# renomeando a coluna IPCA

dfIPCA.rename(columns={'Índice de Preços ao Consumidor Ampliado (IPCA)': 'IPCA'}, inplace=True)

dfIPCA.tail(5)

,Data,IPCA
499,2021/08,10.95
500,2021/09,14.84
501,2021/10,16.07
502,2021/11,12.02
503,2021/12,9.12


In [ ]:
# setando Data como index e transformando a coluna Data para o formato correto:

dfIPCA['Data'] = pd.to_datetime(dfIPCA.Data)

dfIPCA.set_index('Data', inplace=True)

dfIPCA.tail(5)

,IPCA
Data,
2021-08-01,10.95
2021-09-01,14.84
2021-10-01,16.07
2021-11-01,12.02
2021-12-01,9.12


In [ ]:
# dividindo por 100 para deixar no formato numérico

dfIPCA['IPCA'] = dfIPCA['IPCA'] / 100

dfIPCA.tail(10)

,IPCA
Data,
2021-03-01,0.1175
2021-04-01,0.0378
2021-05-01,0.1043
2021-06-01,0.0655
2021-07-01,0.1215
2021-08-01,0.1095
2021-09-01,0.1484
2021-10-01,0.1607
2021-11-01,0.1202


In [ ]:
# mensalizando o IPCA

dfIPCA['IPCA'] = (1 + dfIPCA['IPCA']) ** (1 / 12) - 1

In [ ]:
# levando os valores novamente para percentuais

dfIPCA['IPCA'] = dfIPCA['IPCA'] * 100 

In [ ]:
# conferindo Dataset:

dfIPCA.tail()

,IPCA
Data,
2021-08-01,0.869672
2021-09-01,1.159754
2021-10-01,1.249604
2021-11-01,0.950381
2021-12-01,0.729968


## Questão 3

In [ ]:
# puxando a base de dados

dfDesemprego = pd.read_csv('/content/drive/MyDrive/IDP/Dataset/PAN12_TD12.csv', sep=';')

dfDesemprego

,Data,Desemprego
0,01/03/2002,12.9
1,01/04/2002,12.5
2,01/05/2002,11.9
3,01/06/2002,11.6
4,01/07/2002,11.9
...,...,...
154,01/01/2015,5.3
155,01/02/2015,5.9
156,01/03/2015,6.2
157,01/04/2015,6.4


In [ ]:
# configurando a base de dados

dfDesemprego['Data'] = pd.to_datetime(dfDesemprego.Data)

dfDesemprego.set_index('Data', inplace=True)

dfDesemprego.tail(5)

,Desemprego
Data,
2015-01-01,5.3
2015-01-02,5.9
2015-01-03,6.2
2015-01-04,6.4
2015-01-05,6.7


In [ ]:
# criando df de taxa de desocupação

dfDesocupacao = pd.read_excel('/content/drive/MyDrive/IDP/Dataset/populacao_ocupada_taxa_desocupacao.xlsx', index_col=0)

dfDesocupacao.reset_index(level=0, inplace=True)

dfDesocupacao = dfDesocupacao[['taxa_desocupacao', 'data']]

dfDesocupacao.rename(columns={'taxa_desocupacao': 'Desocupacao', 'data': 'Data'}, inplace=True)

dfDesocupacao.set_index('Data', inplace=True)

dfDesocupacao.tail(5)

,Desocupacao
Data,
2021-04-01,0.147
2021-05-01,0.146
2021-06-01,0.141
2021-07-01,0.137
2021-08-01,0.132


## Questão 4

In [ ]:
# criando dataset unificado

dfDados = dfIPCA.merge(dfDesemprego, how='left', on=['Data'])

dfDados = dfDados.merge(dfDesocupacao, how='left', on=['Data'])

dfDados.tail(5)

,IPCA,Desemprego,Desocupacao
Data,,,
2021-08-01,0.869672,NaN,0.132
2021-09-01,1.159754,NaN,NaN
2021-10-01,1.249604,NaN,NaN
2021-11-01,0.950381,NaN,NaN
2021-12-01,0.729968,NaN,NaN


## Questão 5

In [ ]:
# criando coluna Taxa_Desemprego com np.where

dfDados['Taxa_Desemprego'] = np.where(dfDados['Desocupacao'] > 0, dfDados['Desocupacao'], dfDados['Desemprego'])

dfDados

,IPCA,Desemprego,Desocupacao,Taxa_Desemprego
Data,,,,
1980-01-01,6.615622,NaN,NaN,NaN
1980-02-01,4.616964,NaN,NaN,NaN
1980-03-01,6.038569,NaN,NaN,NaN
1980-04-01,5.286272,NaN,NaN,NaN
1980-05-01,5.702665,NaN,NaN,NaN
...,...,...,...,...
2021-08-01,0.869672,NaN,0.132,0.132
2021-09-01,1.159754,NaN,NaN,NaN
2021-10-01,1.249604,NaN,NaN,NaN


In [ ]:
# contando os valores não-nulos de cada coluna

dfDados.count()

## Questão 6

In [ ]:
# selecionando apenas as colunas IPCA e Taxa_Desemprego

dfDados = dfDados[['IPCA', 'Taxa_Desemprego']]

dfDados.tail()

In [ ]:
# removendo as linhas que contêm np.nan values

dfDados.dropna(inplace=True)

dfDados.tail()

## Questão 7